In [2]:
!pip install autogluon

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 60.4/60.4 kB ? eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.2 kB ? eta -:--:--
     ---------------------------------------- 57.2/57.2 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/130.7 kB ? eta -:--:--
     -------------------------------------- 130.7/130.7 kB 8.0 MB/s eta 0:00:00
  Using cached jsonschema-4.21.1-py3-none-any.whl.metadata (7.8 kB)
     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ---------------------------------------- 43.6/43.6 kB ? eta 0:00:00
  Preparing metadata (setup.py): 

In [3]:
from autogluon.timeseries import TimeSeriesPredictor,TimeSeriesDataFrame
import pandas as pd
import numpy as np

c:\Users\hiro7\anaconda3\envs\fusion_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
